In [ ]:
!pip install langchain_pinecone

In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain pinecone

In [3]:
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone import ServerlessSpec
from langchain.prompts import PromptTemplate
import os

/usr/local/lib/python3.12/dist-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


In [5]:
amazon_data = pd.read_csv('/content/amazon_products_sales_data_cleaned.csv')

In [6]:
amazon_data.columns

Index(['product_title', 'product_rating', 'total_reviews',
       'purchased_last_month', 'discounted_price', 'original_price',
       'is_best_seller', 'is_sponsored', 'has_coupon', 'buy_box_availability',
       'delivery_date', 'sustainability_tags', 'product_image_url',
       'product_page_url', 'data_collected_at', 'product_category',
       'discount_percentage'],
      dtype='object')

In [8]:
unclenad_df = amazon_data[['product_title', 'product_rating', 'discounted_price', 'original_price', 'product_image_url','product_category','product_page_url']]

In [9]:
amazon_data.count()

,0
product_title,42675
product_rating,41651
total_reviews,41651
purchased_last_month,32164
discounted_price,40613
original_price,40613
is_best_seller,42675
is_sponsored,42675
has_coupon,42675
buy_box_availability,28022


In [10]:
filtered_df = unclenad_df[unclenad_df["discounted_price"].notnull() & unclenad_df["original_price"].notnull() & unclenad_df["product_page_url"].notnull()]


In [11]:
filtered_df.count()

,0
product_title,38609
product_rating,37587
discounted_price,38609
original_price,38609
product_image_url,38609
product_category,38609
product_page_url,38609


In [12]:
filtered_df.fillna(0,inplace = True)
df = filtered_df

/tmp/ipython-input-240647582.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.fillna(0,inplace = True)


In [ ]:
openAI_key = ''
pincone_key = ''

In [14]:
pc = Pinecone(api_key=pincone_key)
index_name = "product-recommendation-updated"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        # dimension of the vector embeddings produced by gemini-embedding-001
        dimension=1536, #dimension=768-small, 1536-medium, 3072-large
        metric="cosine",
        # parameters for the free tier index
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
)

# Initialize index client
index = pc.Index(name=index_name)

In [15]:
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=openAI_key)

/tmp/ipython-input-147238453.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=openAI_key)


In [ ]:
df.head(2)

,product_title,product_rating,discounted_price,original_price,product_image_url,product_category
0,BOYA BOYALINK 2 Wireless Lavalier Microphone f...,4.6,89.68,159.00,https://m.media-amazon.com/images/I/71pAqiVEs3...,Phones
1,"LISEN USB C to Lightning Cable, 240W 4 in 1 Ch...",4.3,9.99,15.99,https://m.media-amazon.com/images/I/61nbF6aVIP...,Laptops


In [16]:
documents = []
for _, row in df.iterrows():
    content = f"{row['product_title']} | Category: {row['product_category']} | Rating: {row['product_rating']} | Price: ₹{row['discounted_price']} (was ₹{row['original_price']})"
    metadata = {
        "title": row['product_title'],
        "category": row['product_category'],
        "rating": row['product_rating'],
        "discounted_price": row['discounted_price'],
        "original_price": row['original_price'],
        "image_url": row['product_image_url'],
        "product_url": row['product_page_url']
    }
    documents.append(Document(page_content=content, metadata=metadata))

In [17]:
vectorstore = PineconeVectorStore(index=index, embedding=embedding_model)

In [18]:
vectorstore.add_documents(documents)

['02779815-0e85-472c-a37a-778264b619da',
 '66fb27b7-0e24-4e8c-ae8e-8999733ef07e',
 'beb2512a-ec17-42e1-831e-dd6b509f529e',
 'b72f32ee-e33d-4a9c-8150-edbaa99f4330',
 '350eb183-a0a2-4e96-b94d-93fa73817460',
 '6def691d-20e7-4032-bcd9-ea1d903d7c8b',
 'ea4e0af1-dba7-4321-843e-34ff2a3a7364',
 '23f87bf7-606f-45ee-8464-0d4928874b70',
 '8d918dca-0f6b-43d3-a467-282e72dc7bc2',
 '6a357297-caa8-4388-8040-0ed48a8a27f9',
 '1c6cbfb1-d15e-44f6-a2df-a8f57458553c',
 '2cf6fe4e-3f76-4127-adee-ef5e81e8c075',
 '5cb229e2-aaf4-4982-b34b-cb0e363d500e',
 '9562cb89-8fe1-4439-b893-2bb2551d6eb0',
 'b8dd3456-4c09-47aa-98ed-bda2a5cbdc1d',
 'f512bd67-96b9-4c38-ae0e-d19e1e130e40',
 '61214d61-5fa6-45e2-86b4-bb6edf6fd43c',
 '27fd5157-5e54-49f8-bd2c-fd7ca2b0c37c',
 'cf589049-65f9-4d1e-8fe3-4631ca53c32b',
 '93bbbda5-c939-4602-9f42-37cbc773daaa',
 '4ae0d45e-8c86-4ba8-9857-38acbb527e96',
 '7584884a-e508-417a-8878-a6295b29da35',
 '8393bd11-05c7-47d3-a7e9-97b70cb9389d',
 'd2104e9f-3da0-413e-b610-03b89bec3cdb',
 'a6cb3a58-9a0e-

In [ ]:
# Initialize LLM
llm = ChatOpenAI(model_name="gpt-4", temperature=0.7, api_key = openAI_key)

In [ ]:
query = "show me some cool kitchen products"
retrieved_docs = vectorstore.similarity_search(query, k=5)

In [ ]:
product_list = "\n".join([
    f"{doc.metadata['title']} - ₹{doc.metadata['discounted_price']}, Rating: {doc.metadata['rating']}, Image: {doc.metadata['image_url']}"
    for doc in retrieved_docs
])

In [ ]:
template = PromptTemplate.from_template("""
User query: "{query}"

Products:
{products}

Generate a recommendation in a friendly tone.
""")

prompt = template.format(query=query, products=product_list)

In [ ]:
print (prompt)


User query: "Suggest best laptop under $1000 with good ratings"

Products:
Lenovo IdeaPad 2025 Laptop, 15.6” FHD Anti-Glare, Intel 4-Core CPU, 20GB RAM, 1TB SSD, Wi-Fi 6, 10HR Battery, Win11, 1 Yr ADP + Office 365 w/1TB Cloud Storage, Student & Home Use, w/Cooler Bundle - ₹499.0, Rating: 4.1, Image: https://m.media-amazon.com/images/I/71X3HoaOzzL._AC_UL320_.jpg
Lenovo IdeaPad 1 Student Laptop, Intel Dual Core Processor, 20GB RAM, 1TB SSD + 128GB eMMC, 15.6" FHD Display, 1 Year Office 365, Windows 11 Home, Wi-Fi 6, Webcam, Bluetooth, SD Card Reader, Grey - ₹409.0, Rating: 4.1, Image: https://m.media-amazon.com/images/I/715Hn1Gcv7L._AC_UL320_.jpg
LENOVO IdeaPad 1 Laptop, 15.6” FHD Display, Intel Celeron N4500 Processor, 20GB RAM, 1TB SSD, SD Card Reader, Numeric Keypad, HDMI, Wi-Fi 6, Windows 11 Home, 1 Year Office 365, Grey - ₹335.0, Rating: 4.2, Image: https://m.media-amazon.com/images/I/71496nJIhVL._AC_UL320_.jpg
Lenovo IdeaPad 2025 Family Laptop, 15.6" FHD Screen, Intel N6000 CPU, 3

In [ ]:
response = llm.predict(prompt)
print("Recommendation:\n", response)

Recommendation:
 Sure! Here are some cool kitchen products that you might love:

1. [Melissa & Doug Stainless Steel Pots and Pans Pretend Play Kitchen Set for Kids (8 pcs)](https://www.amazon.com/dp/B00CPHX6YI/) - This set is not only fun for kids but also very durable. It's priced at just ₹17.99 and has a fantastic rating of 4.9! Check it out [here](https://m.media-amazon.com/images/I/712N3bllGLL._AC_UL320_.jpg).
   
2. [Cuisinart MSC-600 3-In-1 Cook Central 6-Quart Multi-Cooker](https://www.amazon.com/dp/B008YEXC22/) - This versatile product can serve as a slow cooker, brown/saute, and steamer. It's priced at ₹193.56 and has a solid rating of 4.4. Take a peek [here](https://m.media-amazon.com/images/I/71vmTdd+gRL._AC_UL320_.jpg).

3. [Microplane Premium Classic Series Zester 18/8, Black](https://www.amazon.com/dp/B00151W38G/) - A great tool for your kitchen, perfect for zesting lemons or grating cheese. It's a steal at just ₹14.99 and has an impressive rating of 4.8. Check it out [he

In [ ]:
from IPython.display import Image, displa
for doc in retrieved_docs:

  url = doc.metadata['image_url']

  display(Image(url=url))
  print(doc.metadata['title'].split(",")[0])

Melissa & Doug Stainless Steel Pots and Pans Pretend Play Kitchen Set for Kids (8 pcs)


Cuisinart MSC-600 3-In-1 Cook Central 6-Quart Multi-Cooker: Slow Cooker


Microplane Premium Classic Series Zester 18/8


Melissa & Doug Wooden Make-a-Cake Mixer Set (11 pcs) - Play Food and Kitchen Accessories - FSC Certified


KitchenAid 9-Speed Digital Hand Mixer with Turbo Beater II Accessories and Pro Whisk - White
